# Segmenting and Clustering Neighborhoods in Toronto

 *This project includes the scraping a wikipedia page to figureout the postalcodes of Canada, then process and clean the data for clustering.Here clustering is done by K-Means and then all clusters are plotted using a library called Folium*.

### Importing and Installing all the required Libraries

In [7]:
import pandas as pd
import numpy as np
import requests
import random
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from  geopy.geocoders import Nominatim
from IPython.display import Image
from IPython.core.display import HTML
from IPython.display import display_html
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans

## Scraping the webpage for the tabel of postal codes, Canada

 _Python uses a library called __BeautifulSoup__ for web scraping the table from wikipedia. To check whether the page has been successfully scraped, the title of the webpage is printed and also the table of postalcodes of Canada is printed_.

In [8]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'lxml')
print(soup.title)
from IPython.display import display_html
tab = str(soup.table)
#display_html(tab,raw=True)

<title>List of postal codes of Canada: M - Wikipedia</title>


### Convert HTML table to Pandas DataFrame

 _For the purpose of cleaning and preprocessing the data we have to convert the HTML table into pandas DataFrame_

In [9]:
canada_data = pd.read_html(tab)
canada = canada_data[0]
print(canada.shape)
canada.head()

(180, 3)


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Preprocessing and Cleaning the Data

 *Before Preprocessing the data in the dataframe 'canada', we copy it into another dataframe 'df' using the function __copy()__, and then the 'Not assigned' values are droped using the functions __replace()__ and __dropna()__. If any values in the field 'Neighborhood' are 'Not assigned', they are replaced with the names of 'Borough'.The dimensions of the preprocessed dataframe 'df' are displayed using __shape__*.

In [10]:
df = canada.copy()

# Replace 'Not assigned' to Nan and Drop the 'NaN' values of 'Borough'
df.replace("Not assigned", np.NaN, inplace=True)
df.dropna(subset = ["Borough"], axis = 0, inplace=True)
df.reset_index(drop = True ,inplace = True)
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood'])
print(df.head())
df.shape

  Postal Code           Borough                                 Neighborhood
0         M3A        North York                                    Parkwoods
1         M4A        North York                             Victoria Village
2         M5A  Downtown Toronto                    Regent Park, Harbourfront
3         M6A        North York             Lawrence Manor, Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government


(103, 3)

## Import the CSV file that contains the coordinates of neighborhoods in Canada.

In [14]:
lat_lon = pd.read_csv('http://cocl.us/Geospatial_data')
lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merging Tables

 *For getting the Latitudes and Longitudes for various neighborhoods in Canada we are merging two tables 'df' and 'lat_lon'*. 

In [18]:
new_df = pd.merge(df, lat_lon, on= 'Postal Code')
new_df.head(15)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
